file_paths = [
    'verified-smart-contracts/test/part.0.parquet',
    'verified-smart-contracts/validation/part.0.parquet',
    'verified-smart-contracts/train/part.0.parquet',
    'verified-smart-contracts/train/part.1.parquet',
    'verified-smart-contracts/train/part.2.parquet',
    'verified-smart-contracts/train/part.3.parquet',
    'verified-smart-contracts/train/part.4.parquet',
]

df = pd.concat([pd.read_parquet(file) for file in file_paths], ignore_index=True)
results = df[df['source_code'].str.contains("block.number|block.timestamp")]
results = results.drop_duplicates(subset='contract_address', keep='first')
results.reset_index(drop=True, inplace=True)
results['contract_address'].to_csv('potential_tl_contract_addresses.csv', index_label='index')

In [17]:
import pandas as pd

In [18]:
# Connect to db
from sqlalchemy import create_engine, text
from utils.db import get_inspect_database_uri

conn = create_engine(get_inspect_database_uri()).connect()

In [21]:
query = f"SELECT * FROM contracts_info"
addresses_df = pd.DataFrame(conn.execute(text(query)))

In [22]:
addresses_df

,contract_address,eth_balance,largest_tx_hash,largest_tx_block_number,largest_tx_value
0,0x4E568Fa05bD656F358f572bE663bcC5dfC9162ae,2.796505e-02,None,None,None
1,0xEb1d77cD6D9EF2F755Bb78e531bfB666E2C24B50,2.473963e-02,None,None,None
2,0x930026539b3eA3D67d3636852b0C350c2A2341b0,1.000000e-18,None,None,None
3,0x30983b29a00fFEe6A7aE94e9715A6E3Bec63d72c,4.057586e-02,None,None,None
4,0x3A3427cb9cE4c8FE213768cca4c2cB050FACfCc2,6.177598e-05,None,None,None
...,...,...,...,...,...
14446,0x8C008283E6194ed0d49B2E9EABA161Ef81615a61,6.000000e-05,None,None,None
14447,0xaB487Ed6A4519545E0840EF8E300bB049f6f45a9,3.000000e-02,None,None,None
14448,0xbb2Ae3e7abF8E268845bADbE7dF18d07D239C2DB,1.700000e-17,None,None,None
14449,0x7D12B8F36897422e2297B43b37cd8039FF9cC3B4,8.423135e-03,None,None,None


# get the 10 highest balance contracts
addresses_df.sort_values(by='eth_balance', ascending=False).head(50)

BASE_GAS_FEE = 11e9 # 11 Gwei
BLOCK_GAS_LIMIT = 30e6 # 30 million units
ETH_TO_WEI = 1e18 # 1 ETH = 1e18 WEI    
# get the contracts whose eth balance is higher than block gas limit * base gas price
addresses_df[addresses_df['eth_balance'] > ((BLOCK_GAS_LIMIT * BASE_GAS_FEE) / ETH_TO_WEI)]
addresses_df.describe()

In [25]:
# get the 10 highest balance contracts
addresses_df.sort_values(by='eth_balance', ascending=False).head(50)

BASE_GAS_FEE = 11e9 # 11 Gwei
BLOCK_GAS_LIMIT = 30e6 # 30 million units
ETH_TO_WEI = 1e18 # 1 ETH = 1e18 WEI    
# get the contracts whose eth balance is higher than block gas limit * base gas price
suff_balance_addresses_df = addresses_df[addresses_df['eth_balance'] > ((BLOCK_GAS_LIMIT * BASE_GAS_FEE) / ETH_TO_WEI)]

In [26]:
addresses_df.describe()

,eth_balance
count,1.445100e+04
mean,1.425956e+00
std,4.175170e+01
min,1.000000e-18
25%,1.400000e-17
50%,3.000000e-03
75%,3.214236e-02
max,4.203247e+03


In [29]:
suff_balance_addresses_df.reset_index(drop=True, inplace=True)
suff_balance_addresses_df

,contract_address,eth_balance,largest_tx_hash,largest_tx_block_number,largest_tx_value
0,0x0cA7dE30FCb509F0de273dF5C1807EA10F28cD3d,1.034894,None,None,None
1,0xB8a5465BFC06fc8C82385dCFf949673D7b068D1a,6.556347,None,None,None
2,0x2F9B2eC511D3C6765Fa2a5De1B29033F15a6A0D5,170.880000,None,None,None
3,0x4c5A30D74F4D7Dbf815a8141e466BE26721C143e,0.500000,None,None,None
4,0xfF74698A258c7B29F0C5aFb578e53e560c5012B9,7.000000,None,None,None
...,...,...,...,...,...
1082,0x557465Ff5869A1Aa8F41ee424C13a04104225494,0.506000,None,None,None
1083,0x97359b3898BE13D71D0817e63D0B43F5D75C7C20,0.500392,None,None,None
1084,0x7d75AB6D8d6294AAa56478b0Eaf3ceB655E25cFC,3.100000,None,None,None
1085,0x8C96EdC82d111E3c5686F5ABE738A82d54d0b887,328.658514,None,None,None


import requests

def fetch_latest_contract_transactions(contracts_address: str, start_block: int, end_block: int, api_key: str):
    url =  f"https://api.etherscan.io/api?module=account&action=txlist&address={contracts_address}&startblock={start_block}&endblock={end_block}&page=1&offset=10000&sort=desc&apikey={api_key}"
    response = requests.get(url)
    return response.json()['result']

OLDEST_BLOCK = 15649595  # first block on October 2022
NEWEST_BLOCK = 18375639  # last block on October 2023
API_KEY = "T48DJNQHTTC6TGKR4ZW19ZSBR9XH17R5S3"
address = addresses_df.sort_values(by='eth_balance', ascending=False).iloc[0]['contract_address']
transactions = fetch_latest_contract_transactions(address, OLDEST_BLOCK, NEWEST_BLOCK, API_KEY)

In [5]:
verified_contract_addresses = pd.read_csv('potential_tl_contract_addresses.csv')['contract_address'].tolist()

In [9]:
# Check if all the verified smart contracts are in the addresses_df
not_available = [""]*len(verified_contract_addresses)
i = 0
for address in verified_contract_addresses:
    if address not in addresses_df['contract_address'].tolist():
        not_available[i] = address
        i += 1

In [16]:
addresses_df['contract_address']

0         0x53424a11FADeadF4fDBB7279b3bcD43F552Ae6a8
1         0x554BC8CBe6B48B6ae4C2b43DdBb691Ecc55F7cEB
2         0xFb4270061d86189776A2dAFea877968DfbE9504b
3         0xC66c050440440FEF0319eb5fc0060Ee97f1d9eB8
4         0x0E6483423a46741bC4a2d2eA63Bd295e3cb12879
                             ...                    
216053    0x930Ce9D2A6F31D41D0caCa15CA653FF0dfb8Cb36
216054    0x99384b884e572d0B1fF309B63A291850C4D9Bf97
216055    0x7515cd37707c700a0bd17D3E925b1b94f2C9B83f
216056    0x48f0572C5898E740Cfdb18efb276f19653aB9564
216057    0xDB03f9ecA02CFFefBb33acDE1a42F78b0eB0Ebd3
Name: contract_address, Length: 216058, dtype: object